In [1]:
import ase
from ase import Atoms
from ase.visualize import view
import numpy as np

In [42]:
def get_chemical_formula(A,B,X,n_array):
    
    x = str(np.sum(n_array+1))
    y = str(np.sum(n_array))
    z = str(np.sum(3*(n_array)+1))

    return A+x+B+y+X+z

def perovskite_atoms(A,B,X,a,b,c,alpha,beta,gamma):
    
    return Atoms([A,B,X,X,X], 
           scaled_positions=[(0.5,0.5,0.5),(0,0,0),(0.5,0,0),(0,0,0.5),(0,0.5,0)],
           cell=[a,b,c,alpha,beta,gamma])

def rocksalt_atoms(A,X,a,b,c,alpha,beta,gamma):
    
    return Atoms([A,X], 
           scaled_positions=[(0,0,0),(0.5,0.5,0)],
           cell=[a,b,c,alpha,beta,gamma])

def create_rp_structure_single(A,B,X,n_array,a,b,perovskite_spacing,rocksalt_spacing):

    total_length = 2*np.sum(n_array)*perovskite_spacing + len(n_array)*1*perovskite_spacing 
    
    rp_structure = Atoms(cell=[a,b,total_length,90,90,90],pbc=[True,True,True])

    perovskite_layer = perovskite_atoms(A,B,X,a,b,perovskite_spacing,90,90,90)
    rocksalt_layer = rocksalt_atoms(A,X,a,b,rocksalt_spacing,90,90,90)

    for number_of_layers in n_array:
        
        rp_structure += perovskite_layer
        perovskite_layer.translate(np.dot([0,0,1],perovskite_layer.cell))
        rocksalt_layer.translate(np.dot([0,0,1],perovskite_layer.cell))
        
        rp_structure += rocksalt_layer
        perovskite_layer.translate(np.dot([0.5,0.5,0.5],perovskite_layer.cell))
        
        for _ in range(number_of_layers):
            rp_structure += perovskite_layer
            perovskite_layer.translate(np.dot([0,0,1],perovskite_layer.cell))
            rocksalt_layer.translate(np.dot([0,0,1],perovskite_layer.cell))
            
        rocksalt_layer.translate(np.dot([0.5,0.5,0.5],perovskite_layer.cell))
        rp_structure += rocksalt_layer
        rocksalt_layer.translate(np.dot([-0.5,-0.5,0.5],perovskite_layer.cell))
        perovskite_layer.translate(np.dot([-0.5,-0.5,0.5],perovskite_layer.cell))
        
        for _ in range(number_of_layers-1):
            rp_structure += perovskite_layer
            perovskite_layer.translate(np.dot([0,0,1],perovskite_layer.cell))
            rocksalt_layer.translate(np.dot([0,0,1],perovskite_layer.cell))
        
    return rp_structure


def create_rp_structure_mixed(A,B,X,n_array,a,b,perovskite_spacing,rocksalt_spacing):

    total_length = 1*np.sum(n_array)*perovskite_spacing + len(n_array)*0.5*perovskite_spacing 
    
    rp_structure = Atoms(cell=[a,b,total_length,90,90,90],pbc=[True,True,True])

    perovskite_layer = perovskite_atoms(A,B,X,a,b,perovskite_spacing,90,90,90)
    rocksalt_layer = rocksalt_atoms(A,X,a,b,rocksalt_spacing,90,90,90)

    for number_of_layers in n_array:
        
        for _ in range(number_of_layers):
            rp_structure += perovskite_layer
            perovskite_layer.translate(np.dot([0,0,1],perovskite_layer.cell))
            rocksalt_layer.translate(np.dot([0,0,1],perovskite_layer.cell))
        
        rp_structure += rocksalt_layer
        rocksalt_layer.translate(np.dot([0.5,0.5,0.5],perovskite_layer.cell))
        perovskite_layer.translate(np.dot([0.5,0.5,0.5],perovskite_layer.cell))
        
        
    # need to think about what happens at PBC

    return rp_structure

def rattle_structure(atoms_object):

    return atoms_object.rattle(stdev=0.02, seed=1)

def save_structure(atoms_object,n_array,A,B,X):

    ase.io.write('RP_{}_{}.cif'.format(''.join(map(str,n_array)),A+B+X), atoms_object)

In [46]:
atoms = create_rp_structure_single('Ba','Zr','S',np.array([8]),5,5,5,1)
save_structure(atoms,np.array([8]),"Ba","Zr","S")

In [47]:
atoms = create_rp_structure_mixed('Ba','Zr','S',np.array([8]),5,5,5,1)
save_structure(atoms,np.array([8,3]),"Ba","Zr","S")

In [5]:
for atom in atoms:
    print(atom)
    print(atom.scaled_position)

Atom('Ba', [2.022977511558638, 2.022977511558638, 2.407259664232843], index=0)
[0.5        0.5        0.16666667]
Atom('Zr', [0.0, 0.0, 0.0], index=1)
[0. 0. 0.]
Atom('S', [2.5, 2.5, 0.0], index=2)
[0.5 0.5 0. ]
Atom('S', [2.022977511558638, -0.477022488441362, 2.407259664232843], index=3)
[ 5.00000000e-01 -1.11022302e-17  1.66666667e-01]
Atom('S', [-0.477022488441362, 2.022977511558638, 2.407259664232843], index=4)
[-1.11022302e-17  5.00000000e-01  1.66666667e-01]
Atom('Ba', [-2.747247372854982, -2.747247372854982, 26.479856306561274], index=5)
[0.5        0.5        1.83333333]
Atom('Zr', [-4.77022488441362, -4.77022488441362, 24.07259664232843], index=6)
[-1.77635684e-16 -1.77635684e-16  1.66666667e+00]
Atom('S', [-2.2702248844136204, -2.2702248844136204, 24.07259664232843], index=7)
[0.5        0.5        1.66666667]
Atom('S', [-2.747247372854982, -5.247247372854982, 26.479856306561274], index=8)
[ 5.00000000e-01 -1.77635684e-16  1.83333333e+00]
Atom('S', [-5.247247372854982, -2.74

In [45]:
atoms.cell = ase.Atoms.cell([5,5,11,101,101,90],pbc=[True,True,True])

NameError: name 'a' is not defined

In [4]:
t = get_chemical_formula("Ba","Zr","S",np.array([2,1]))

In [38]:
perov = perovskite_layer("Ba","Zr","S")
view(perov, viewer='ngl')

In [5]:
t

'Ba5Zr3S11'

In [39]:
rs = rocksalt_layer("Ba","S")
view(rs, viewer='ngl')

In [27]:
atoms


Atoms(symbols='BaZrS3BaS', pbc=True, cell=[[5.0, 0.0, 0.0], [0.0, 5.0, 0.0], [-1.5264719630123584, -1.5264719630123584, 7.703230925545099]])

In [8]:
t.translate([2,0,0])

In [12]:
perovskite_layer_shifted = perovskite_layer('Ba','Zr','S').translate([0,0,current_c_position])

NameError: name 'current_c_position' is not defined